In [10]:
-- imports
import qualified Data.Char as Char
import qualified Data.List as List

# 1장 functional programming은 무엇일까?

한 마디로 얘기하면:

 * functional programming은 명령과 명령의 실행보다 함수와 함수의 적용을 강조하는 프로그래밍 방법이다. 명령과 함수는 side-effect의 차이로 구분할 수 있다.
 * functional programming은 문제를 명확하고 간결하게 표현할 수 있는 단순한 수학 표기방법을 사용한다.
 * functional programming은 프로그램의 속성들에 대해서 등식추론(equational reasoning)을 지원하는 단순한 수학적 기반을 가진다. (추가 설명 필요???)
 
## 1.1 Functions and types

Haskell에서는 아래와 같이 함수의 형을 표기한다.

    -- 함수 f는 1개의 X형의 입력을 받아서 Y형의 결과를 반환한다.
    f :: X -> Y

    -- 다른 예들
    sin :: Float -> Float
    age :: Person -> Int


## 1.2 Functional composition
f :: Y -> Z, g :: X -> Y의 두 함수가 주어졌을때 우리는 이들을 합쳐서 새로운 함수를 만들 수 있다. (composition)

    f . g :: X -> Z

이 새로운 함수는 아래와 같아진다.

    (f . g) x = f (g x)


## 1.3 Example: common words



In [11]:
type Text = [Char]
type Word = [Char]

-- map :: (a->b) -> [a] -> [b] from Prelude
-- toLower :: Char -> Char from Data.Char
-- map toLower :: Text -> Text

sortWords :: [Word] -> [Word]
sortWords = List.sort

countRuns :: [Word] -> [(Int,Word)]
countRuns [] = []
countRuns (x:xs) = (hl, x) : countRuns t
                   where hl = 1 + (length . takeWhile (\v -> x==v)) xs
                         t = dropWhile (\v -> x==v) xs
                         
sortRuns :: [(Int,Word)] -> [(Int,Word)]
sortRuns = List.reverse . List.sort

-- take :: Int -> [a] -> [a]

showRun :: (Int,Word) -> String
showRun (x,y) = concat [y, "\t", show(x), "\n"]

-- map showRun :: [(Int,Word)] -> [String]

-- concat :: [[a]] -> [a] from Prelude
commonWords :: Int -> Text -> String
commonWords n = concat . map showRun . take n . 
                sortRuns . countRuns . sortWords . 
                words . map Char.toLower

sample = "cc ba aa ba cc cc"

putStrLn $ commonWords 2 sample

cc	3
ba	2

commonWords 함수는 functional composition에 의해서 8개의 함수들의 파이프라인으로 정의된다. 모든 문제를 이와 같이 바로 분해할 수는 없겠지만 이와 같이 문제를 분해하여 얻어진 프로그램은 단순하며 아름답고 효과적이다.

문제를 구성하는 종속적인 함수들의 자료형에 따라서 어떻게 문제가 분해되는지를 잘 보자. 첫번째 가르침이 functional composition의 중요성이라면, 두번째 중요한 가르침은 함수의 자료형을 결정하는 것은 함수의 올바른 정의를 찾아내는 첫 단계라는 것이다.

우리가 commonWords를 정의한 방법은 이것을 구성하는 종속적인 함수의 정의를 작성한 것이다. 우리는 이러한 정의들은 'script'라 부른다. script의 순서는 중요하지 않다. 따라서 우리는 commonWords를 제일 앞에 둘 수 있었다.

## 1.4 Example: numbers into words

이 예제에서는 복잡한 문제들을 단순화하고 단순화된 문제를 해결하는 문제풀이 방식을 보여준다.
이 예제에서는 아래와 같이 숫자를 입력받아서 영어 단어들로 변환하는 함수를 작성해본다.

    convert 308000 = "three hundred and eight thousand"
    convert 369027 = "three hundred and sixty-nine thousand and twenty-seven"

In [12]:
convert :: Int -> String
convert = convert6

units, teens, tens :: [String]
units = ["zero","one","two","three","four","five","six","seve","eight","nine"]
teens = ["ten","eleven","twelve","thirteen","fourteen","fifteen","sixteen","seventeen","eighteen","nineteen"]
tens = ["twenty","thirty","forty","fifty","sixty","seventy","eighty","ninety"]

convert1 :: Int -> String
convert1 n = units !! n

digits2 :: Int -> (Int,Int)
digits2 n = (div n 10, mod n 10)
-- You can use infix version of div and mod by using `
-- digits2 n = (n `div` 10, n `mod` 10)
-- You can also use divMod
-- digits2 n = divMod n 10

convert2 :: Int -> String
convert2 = combine2 . digits2

combine2 :: (Int,Int) -> String
combine2 (t,u)
  | t==0 = units!!u
  | t==1 = teens!!u
  -- | 2<=t && u==0 = tens!!(t-2)
  | u==0 = tens!!(t-2)
  -- | 2<=t && u/=0 = tens!!(t-2) ++ "-" ++ units!!u
  | otherwise = tens!!(t-2) ++ "-" ++ units!!u

-- yet another way of writing convert2
convert2' :: Int -> String
convert2' n
  | t==0 = units!!u
  | t==1 = teens!!u
  | u==0 = tens!!(t-2)
  | otherwise = tens!!(t-2) ++ "-" ++ units!!u
  where (t,u) = (n `div` 10, n `mod` 10)

convert3 :: Int -> String
convert3 n
  | h==0 = convert2 t
  | n==0 = units!!h ++ " hundred"
  | otherwise = units!!h ++ " hundred and " ++ convert2 t
  where (h,t) = (n `div` 100, n `mod` 100)

convert6 :: Int -> String
convert6 n
  | m==0 = convert3 h
  | h==0 = convert3 m ++ " thousand"
  | otherwise = convert3 m ++ " thousand" ++ link h ++ convert3 h
  where (m,h) = (n `div` 1000, n `mod` 1000)

link :: Int -> String
link h = if h < 100 then " and " else " "
-- We could also have used guarded equations
-- link h | h < 100 = " and "
--        | otherwise = " "

## 1.6 Exercises

### Exercise A

In [13]:
double :: Integer -> Integer
double x = 2 * x

map double [1,4,4,3]
map (double . double) [1,4,4,3]
map double []

sample2 = [1,2,3,4]
sample3 = [[1,2], [3,4], [5]]

(List.sum . map double) sample2 == (double . List.sum) sample2
(List.sum . map List.sum) sample3 == (List.sum . concat) sample3
(List.sum . List.sort) sample2 == List.sum sample2

[2,8,8,6]

[4,16,16,12]

[]

True

True

True

### Exercise D

In [14]:
(words . map Char.toLower) sample
(map (map Char.toLower) . words) sample

["cc","ba","aa","ba","cc","cc"]

["cc","ba","aa","ba","cc","cc"]

### Exercise F

In [15]:
anagrams :: Int -> [Word] -> String
anagrams n = List.concat . List.intersperse "\n" . map showItem . groupItems . sortItems . map normalize . take n

normalize :: Word -> (Word,Word)
normalize w = (List.sort w, w)

sortItems :: [(Word,Word)] -> [(Word,Word)]
sortItems = List.sort

groupItems :: [(Word,Word)] -> [(Word,[Word])]
groupItems [] = []
groupItems xxs@((n,w):xs) = (n, hl) : groupItems t
                   where hl = (map snd . takeWhile (\(v,vs) -> n==v)) xxs
                         t = dropWhile (\(v,vs) -> n==v) xs

showItem :: (Word,[Word]) -> String
showItem (x,xs) = show . List.concat $ x:": ":(List.intersperse "," xs)

putStr $ anagrams 5 ["abc", "bca", "cab", "def", "fed"]

"abc: abc,bca,cab"
"def: def,fed"

### Exercise G

In [16]:
song :: Int -> String
song n  = if n==0 then ""
          else song (n-1) ++ "\n" ++ verse n

verse :: Int -> String
verse n = line1 n ++ line2 n ++ line3 n ++ line4 n

number = ["","one","two","three","four",
          "five","six","seven","eight","nine"]

man :: Int -> String
man n = number !! n ++ if n == 1 then " man" else " men"

mans :: Int -> String
mans n = (concat . List.intersperse ", " . map man . reverse)  [1..n]

cap :: String -> String
cap [] = []
cap (x:xs) = Char.toUpper x : xs

line1 :: Int -> String
line1 n = cap $ man n ++ " went to mow\n"

line2 :: Int -> String
line2 n = "Went to mow a meadow\n"

line3 :: Int -> String
line3 n = cap $ mans n ++ " and his dog\n"

line4 :: Int -> String
line4 n = "Went to mow a meadow\n"

putStr $ song 1
putStr $ song 2


One man went to mow
Went to mow a meadow
One man and his dog
Went to mow a meadow


One man went to mow
Went to mow a meadow
One man and his dog
Went to mow a meadow

Two men went to mow
Went to mow a meadow
Two men, one man and his dog
Went to mow a meadow